In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from keras.applications.densenet import DenseNet121, preprocess_input
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.utils import Sequence
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

PATH = '/workspace/AI/OpenSource/celebrities/'

celeb_data = pd.read_csv(PATH + 'identity_CelebA.txt', sep=" ", header=None)
celeb_data.columns = ["image", "label"]
celeb_data.head()
attributes = pd.read_csv('/workspace/AI/OpenSource/celebrities/list_attr_celeba.csv')

Using TensorFlow backend.


In [2]:
checkpoint = ModelCheckpoint('attributes.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

attributes = attributes[['image_id','Attractive','Bald','Male','Bags_Under_Eyes','Narrow_Eyes',
                         'Oval_Face','Pointy_Nose','Receding_Hairline','Young']]
images = attributes['image_id']
labels = attributes.drop(['image_id'], axis=1).values
labels[labels<0] = 0
labels = labels.astype('float32')

In [3]:
data = [images, np.arange(len(images))]

df = pd.DataFrame(data).transpose()
df.columns = ['image','id']

datagen = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, fill_mode='nearest',
                             validation_split=0.2, horizontal_flip=True, preprocessing_function=preprocess_input)

train_gen = datagen.flow_from_dataframe(df, directory=PATH+'img_align_celeba', x_col='image', 
                                        y_col='id', target_size=(218, 178), color_mode='rgb', subset = 'training',
                                        classes=None, class_mode='sparse', batch_size=32, shuffle=True)
valid_gen = datagen.flow_from_dataframe(df, directory=PATH+'img_align_celeba', x_col='image', 
                                        y_col='id', target_size=(218, 178), color_mode='rgb', subset = 'validation',
                                        classes=None, class_mode='sparse', batch_size=32, shuffle=True)

densenet = DenseNet121(include_top=False, weights='imagenet', input_shape=(218, 178, 3))
Convfeat = GlobalAveragePooling2D()(densenet.output)
for layer in densenet.layers:
    layer.trainable = False
x = BatchNormalization()(Convfeat)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dense(9, activation='sigmoid')(x)
att_model = Model(densenet.input, x)

def attribute_crossentropy(y_true, y_pred):
    y_true = tf.gather(labels, tf.to_int32(y_true[0]))
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)

att_model.compile(Adam(lr=0.0002), attribute_crossentropy)


att_model.fit_generator(train_gen, steps_per_epoch=len(train_gen), epochs=25, callbacks = [checkpoint], initial_epoch = 0,
                        validation_data=valid_gen, validation_steps=len(valid_gen), use_multiprocessing=True, workers=8)

for layer in att_model.layers:
    layer.trainable = True

att_model.fit_generator(train_gen, steps_per_epoch=len(train_gen), epochs=50, callbacks = [checkpoint], initial_epoch = 25,
                        validation_data=valid_gen, validation_steps=len(valid_gen), use_multiprocessing=True, workers=8)

Found 162080 images belonging to 202599 classes.
Found 40519 images belonging to 202599 classes.
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_194:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_192:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()  File "/usr/lib/python3.5/asyncio/bas

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_190:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()  File "/usr/lib/python3.5/asyncio/bas

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_188:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()  File "/usr/lib/python3.5/asyncio/bas

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_186:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()  File "/usr/lib/python3.5/asyncio/bas

...

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_6:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()  File "/usr/lib/python3.5/asyncio/base_

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_4:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()  File "/usr/lib/python3.5/asyncio/base_

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_2:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()  File "/usr/lib/python3.5/asyncio/base_

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()  File "/usr/lib/python3.5/asyncio/base_ev

KeyboardInterrupt: 